# AgentCore Browser with Session Replay

We can replay browser sessions using the Amazon Bedrock AgentCore SDK to view session recordings stored in Amazon S3. 

This feature enables us to review past browser interactions for debugging, auditing, or training purposes. 

The recordings in S3 include DOM change events, browser network activity, and console logs for comprehensive session analysis.

In [10]:
import json
import boto3
import gzip
import io

region="us-east-1"
role_name = "BrowserS3ExecutionRole"
bucket_name = "browser-tool-recording-bucket"
s3_prefix = "replay_data"
policy_name = "s3_access"


## Create IAM Role With Permission

In [8]:
iam_client = boto3.client('iam')

# Create role
response = iam_client.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock-agentcore.amazonaws.com"
                },
                "Action": "sts:AssumeRole",
            }
        ]
    }),
)
role_arn = response["Role"]["Arn"]
print(f"role created: {role_arn}")

# Add policy
iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName=policy_name,
    PolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "s3:PutObject",
                    "s3:ListMultipartUploadParts",
                    "s3:AbortMultipartUpload"
                ],
                "Resource": f"arn:aws:s3:::{bucket_name}/{s3_prefix}/*"
            }
        ]
    })
)

role created: arn:aws:iam::075198889659:role/BrowserS3ExecutionRole


{'ResponseMetadata': {'RequestId': '59d0060e-32fb-4e63-adb1-8e1530e7c6fd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 05 Aug 2025 09:44:05 GMT',
   'x-amzn-requestid': '59d0060e-32fb-4e63-adb1-8e1530e7c6fd',
   'content-type': 'text/xml',
   'content-length': '206'},
  'RetryAttempts': 0}}

## Create S3 To Store Recording

In [9]:
s3_client = boto3.client(
    's3',
    region_name=region,
)
s3_client.create_bucket(
    Bucket=bucket_name,
    ACL="private",
    # CreateBucketConfiguration={
    #     "LocationConstraint": region # for any region other than us-east-1
    # }
)


{'ResponseMetadata': {'RequestId': 'NFWSCD1QZ0V1YQPQ',
  'HostId': 'e7AhC37Vj9PX7/T/cFrCJWCcNnF8z0udhKX4t6SoqRVkVlR9AGk65bHlyzJq9pWN940N9BwAfE0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'e7AhC37Vj9PX7/T/cFrCJWCcNnF8z0udhKX4t6SoqRVkVlR9AGk65bHlyzJq9pWN940N9BwAfE0=',
   'x-amz-request-id': 'NFWSCD1QZ0V1YQPQ',
   'date': 'Tue, 05 Aug 2025 07:39:45 GMT',
   'location': '/browser-tool-recording-bucket',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/browser-tool-recording-bucket'}

## Create Custom Browser Tool

create a Browser with recording enabled and provide the S3 bucket and prefix where you want the recording to be stored

In [18]:
control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region,
    endpoint_url=f"https://bedrock-agentcore-control.{region}.amazonaws.com"
)

In [10]:
response = control_client.create_browser(
    name="custom_browser_with_s3",
    executionRoleArn=role_arn,
    networkConfiguration={
        "networkMode": "PUBLIC"
    },
    recording={
        "enabled": True,
        "s3Location": {
            "bucket": bucket_name,
            "prefix": f"{s3_prefix}/" # trailing slash has to be included. Otherwise, files will be created as replay_data01K1WKB928JZGX0HXBRW87V9J7 instead of replay_data/01K1WKB928JZGX0HXBRW87V9J7
        }
    }
)
browser_id = response["browserId"]
print(f"browser created: {browser_id}")

browser created: custom_browser_with_s3-mAuMr7efkH


## Run Some Sessions To Get Replay

Session replay in Amazon Bedrock AgentCore captures DOM mutations within your browser session and replays those changes by reconstructing the DOM. During replay, the browser may make cross-origin HTTP requests to fetch external assets such as JavaScript files, CSS stylesheets, images, and other resources required to render the page accurately.

We can also view live session replay directly from the AgentCore Console. 

* This feature is not available in the AWS managed Browser (aws.browser.v1).

In [12]:
from bedrock_agentcore.tools.browser_client import BrowserClient
from playwright.async_api import async_playwright, BrowserType

browser_client = BrowserClient(region=region)
session_id = browser_client.start(
    identifier=browser_id
)
print(f"session started: {session_id}")
ws_url, headers = browser_client.generate_ws_headers()

async with async_playwright() as playwright:
    chromium: BrowserType = playwright.chromium
    browser = await chromium.connect_over_cdp(
        endpoint_url=ws_url,
        headers=headers
    )
    page = await browser.new_page()

    try:
        await page.goto("https://medium.com/@itsuki.enjoy")
        print(await page.title())
        await page.get_by_role('tab', name='About').click()
    finally:
        await page.close()
        await browser.close()
        browser_client.stop()


session started: 01K1WT2P8JMQQWGMY0SE6S4J7V
Itsuki – Medium


## Get Recording From S3

Recording for a specific session id is stored within {bucket_name}/{s3_prefix}/{session_id}/

With the following files: 
- metadata.json: contains metadata of the recording, such as sessionId, totalEvents, recording startTime, and a reference to a list of `ndjson.gz` files that will contain the actual recording events.
- a batch of *.ndjson.gz: specific events including DOM change events, browser network activity, and console logs for comprehensive session analysis.


For visualize the recording events, please refer to the [view_recordings.py](https://github.com/awslabs/amazon-bedrock-agentcore-samples/blob/main/01-tutorials/05-AgentCore-tools/02-Agent-Core-browser-tool/interactive_tools/live_view_sessionreplay/view_recordings.py#L337) provided by AWS.

#### Metadata.json

In [6]:
metadata_key = f"{s3_prefix}/{session_id}/metadata.json"
response = s3_client.get_object(Bucket=bucket_name, Key=metadata_key)
metadata = json.loads(response['Body'].read().decode('utf-8'))
print(f"metadata downloaded:")
print(json.dumps(metadata, indent=2))

metadata downloaded:
{
  "sessionId": "01K1WT2P8JMQQWGMY0SE6S4J7V",
  "startTime": "2025-08-05T09:47:56.885974",
  "endTime": null,
  "status": "recording",
  "batches": [
    {
      "file": "batch-cdp-1-1754387285321.ndjson.gz",
      "batchNumber": 1,
      "eventCount": 845,
      "sizeBytes": 182523,
      "originalSizeBytes": 1315266,
      "compressionRatio": 0.8612273106732783,
      "compressed": true,
      "startTime": "2025-08-05T09:47:56.885974",
      "endTime": "2025-08-05T09:48:05.266774"
    },
    {
      "file": "batch-cdp-2-1754387288085.ndjson.gz",
      "batchNumber": 2,
      "eventCount": 149,
      "sizeBytes": 26336,
      "originalSizeBytes": 248850,
      "compressionRatio": 0.8941691782198111,
      "compressed": true,
      "startTime": "2025-08-05T09:47:56.885974",
      "endTime": "2025-08-05T09:48:08.069115"
    }
  ],
  "totalEvents": 994,
  "compressionEnabled": true,
  "lastUpdated": "2025-08-05T09:48:08.%fZ"
}


#### Process Batches for actual events

In [ ]:
batch_files = [item["file"] for item in metadata["batches"]]
print(f"{len(batch_files)} bathces in recordings: {batch_files}.")

events = []

for item in batch_files:
    try:
        batch_key = f"{s3_prefix}/{session_id}/{item}"
        print(f"Downloading batch file: {batch_key}")
        response = s3_client.get_object(Bucket=bucket_name, Key=batch_key)

        # Try to read as gzipped JSON lines
        with gzip.GzipFile(fileobj=io.BytesIO(response['Body'].read())) as gz:
            content = gz.read().decode('utf-8')
            print(f"Read {len(content)} bytes of content")

            # Process each line as a JSON event
            for line in content.splitlines():
                if line.strip():
                    try:
                        event = json.loads(line)
                        # Validate event
                        if 'type' in event and 'timestamp' in event:
                            events.append(event)
                        else:
                            print(f"Skipping invalid event (missing required fields)")
                    except json.JSONDecodeError as je:
                        print(f"Invalid JSON in line: {line[:50]}...")

            print(f"Added {len(events)} events")
            print("------------")

    except Exception as e:
        print(f"Error processing file {item}: {e}")

print("Processing finished.")
print(f"Loaded {len(events)} events.")
# Each event will be in the following format
# {'type': 6, 'data': {'plugin': 'rrweb/cdp@1', 'payload': {'type': 'cdp-event', 'data': {'timestamp': 1754387277097, 'direction': 'incoming', 'method': 'Browser.getVersion', 'params': None}}}, 'timestamp': 1754387277097}

2 bathces in recordings: ['batch-cdp-1-1754387285321.ndjson.gz', 'batch-cdp-2-1754387288085.ndjson.gz'].
Read 1315266 bytes of content
Added 845 events
------------
Read 248850 bytes of content
Added 994 events
------------
Processing finished.
Loaded 994 events.
[{'type': 6, 'data': {'plugin': 'rrweb/cdp@1', 'payload': {'type': 'cdp-event', 'data': {'timestamp': 1754387277097, 'direction': 'incoming', 'method': 'Browser.getVersion', 'params': None}}}, 'timestamp': 1754387277097}, {'type': 6, 'data': {'plugin': 'rrweb/cdp@1', 'payload': {'type': 'cdp-response', 'data': {'timestamp': 1754387277099, 'direction': 'outgoing', 'id': 1, 'result': {'protocolVersion': '1.3', 'product': 'Chrome/140.0.7265.0', 'revision': '@59f127c941450f603d7ee72282ee71d55a2ec13b', 'userAgent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36', 'jsVersion': '14.0.55'}}}}, 'timestamp': 1754387277099}, {'type': 6, 'data': {'plugin': 'rrweb/cdp@1', 'payload': {

## Clean Up

In [22]:
agent_core = boto3.client(
    'bedrock-agentcore',
    region_name=region,
    endpoint_url=f"https://bedrock-agentcore.{region}.amazonaws.com"
)

response = agent_core.list_browser_sessions(
    browserIdentifier=browser_id,
    status="READY" # 'READY'|'TERMINATED'
)

# stop all the running sessions
for item in response["items"]:
    agent_core.stop_browser_session(
        browserIdentifier=browser_id,
        sessionId=item["sessionId"]
    )

response = control_client.delete_browser(
    browserId = browser_id
)

# delete role policy and role
iam_client.delete_role_policy(
    RoleName=role_name,
    PolicyName=policy_name
)

iam_client.delete_role(
    RoleName = role_name
)
print("Deleted!")

Deleted!
